In [1]:
# 2. データの読み込み（Excelファイル）

from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

file_path = "/content/drive/My Drive/github/利益予測データベース.xlsx"  # アップロードしたファイルのパス
df = pd.read_excel(file_path)

# 1. 必要なライブラリをインポート
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

Mounted at /content/drive


In [2]:
# 3. データ分割
# 'df' (DataFrame) をNumPy配列に変換します
data = df.values

x_train = data[:, 3:8][data[:, 1] == 1]
y_train = data[:, 2][data[:, 1] == 1]
x_test = data[:, 3:8][data[:, 1] == 2]
y_test = data[:, 2][data[:, 1] == 2]

In [3]:
# 4. 設定（ハイパーパラメータ）

MAX_TRIALS = 200 #200

epochs = 5
batch_size = 60

In [4]:
# 5. 型変換
x_train = np.array(x_train, dtype=float)
y_train = np.array(y_train, dtype=float)
x_test = np.array(x_test, dtype=float)
y_test = np.array(y_test, dtype=float)

In [5]:
%%time

from tensorflow.keras import layers, models

# 6. 繰り返し予測
predictions = []
se          = []
mean_score  = []
score       = []

for trial in range(MAX_TRIALS):
    print(f"Trial {trial + 1}/{MAX_TRIALS}")  # 試行回数を表示
# 入力層
    inputs = layers.Input(shape=(x_train.shape[1],))
# 隠れ層
    x = layers.Dense(64, activation='relu')(inputs)
    #x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation='relu')(x)
    #x = layers.Dropout(0.3)(x)

# 出力層（softmax ではなく linear）
    outputs = layers.Dense(1, activation='linear')(x)

# モデル作成
    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mae', metrics=['mae'])

    history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
                        validation_data=(x_test, y_test),
                        verbose=0)

    y_pred = model.predict(x_test,verbose=0).flatten()
    predictions.append(y_pred)
    preds_array = np.array(predictions)

    N = len(predictions)  # 試行回数
    se.append(preds_array.std(axis=0) / np.sqrt(N))  # 各観測値の利益予想の標準誤差を格納

    score.append(model.evaluate(x_test, y_test, verbose=0)[0])
    mean_score.append(np.mean(score))

Trial 1/200
Trial 2/200


KeyboardInterrupt: 

In [6]:
se = np.array(se)
mean_score = np.array(mean_score)

import matplotlib.pyplot as plt

#print(se.shape)
print('各観測値の95％信頼区間の最大:',1.96 * np.max(se, axis = 1)[MAX_TRIALS-1])
print('各観測値の95％信頼区間の平均:',1.96 * np.mean(se, axis = 1)[MAX_TRIALS-1])
#print(mean_score.shape)
print('MAEの平均値:',mean_score[MAX_TRIALS-1])

# Running Test Predictions
plt.figure(figsize=(8, 4))
plt.plot(1.96 *np.max(se,axis = 1), color = "black")
plt.plot(1.96 *np.mean(se,axis = 1), color = "green")
plt.plot(mean_score, color = "red")
plt.legend()#loc='lower right')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.grid(which='major', linestyle='--', axis='both')

plt.show()

IndexError: index 199 is out of bounds for axis 0 with size 1

In [ ]:
# 7. 平均予測を計算して保存
mean_preds = preds_array.mean(axis=0)
output_df = pd.DataFrame({
    'sample': df["sample"][data[:, 1] == 2].values,
    'y_pred': mean_preds
})

In [ ]:
# 8. 保存先を指定（任意の場所に変更可）
save_path = "/content/drive/My Drive/github"
output_df.to_excel(save_path, index=False)
print("予測結果を保存しました：", save_path)